In [1]:
import csv
import pandas as pd
import numpy as np
import processing 
from datetime import datetime as dt
import pylab as pl
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def convert_to_datetime(series_row, date_format):
    if str(series_row) == 'nan':
        return float('nan')
    return dt.strptime(series_row, date_format)
    

def convert_to_month(series_row):
    if str(series_row) == 'NaT' or str(series_row)== 'nan':
        return float('nan')
    else:
        return str(series_row.month)

def convert_to_year(series_row):
    if str(series_row) == 'NaT' or str(series_row)== 'nan':
        return float('nan')
    else:
        return str(series_row.year)

def get_month_year_col(df, date_column, date_format):
    df[date_column+'_datetime'] = df[date_column].apply(convert_to_datetime, date_format=date_format)
    df[date_column+'_month'] = df[date_column+'_datetime'].apply(convert_to_month)
    df[date_column+'_year'] = df[date_column+'_datetime'].apply(convert_to_year)
    return df

In [4]:
violhist = pd.read_csv('data/ICIS-AIR_VIOLATION_HISTORY.csv')

In [5]:
fce = pd.read_csv('data/ICIS-AIR_FCES_PCES.csv')

In [6]:
violhist.columns

Index(['PGM_SYS_ID', 'ACTIVITY_ID', 'AGENCY_TYPE_DESC', 'STATE_CODE',
       'AIR_LCON_CODE', 'COMP_DETERMINATION_UID', 'ENF_RESPONSE_POLICY_CODE',
       'PROGRAM_CODES', 'PROGRAM_DESCS', 'POLLUTANT_CODES', 'POLLUTANT_DESCS',
       'EARLIEST_FRV_DETERM_DATE', 'HPV_DAYZERO_DATE', 'HPV_RESOLVED_DATE'],
      dtype='object')

In [8]:
fce.columns

Index(['PGM_SYS_ID', 'ACTIVITY_ID', 'STATE_EPA_FLAG', 'ACTIVITY_TYPE_CODE',
       'ACTIVITY_TYPE_DESC', 'COMP_MONITOR_TYPE_CODE',
       'COMP_MONITOR_TYPE_DESC', 'ACTUAL_END_DATE', 'PROGRAM_CODES'],
      dtype='object')

In [9]:
violhist = get_month_year_col(violhist, 'EARLIEST_FRV_DETERM_DATE', '%m-%d-%Y')

In [10]:
violhist = get_month_year_col(violhist, 'HPV_DAYZERO_DATE', '%m-%d-%Y')

In [11]:
violhist = get_month_year_col(violhist, 'HPV_RESOLVED_DATE', '%m-%d-%Y')

In [54]:
#removing FRVs
violhist = violhist[violhist.ENF_RESPONSE_POLICY_CODE != 'FRV']

In [55]:
fce = get_month_year_col(fce, 'ACTUAL_END_DATE', '%m-%d-%Y')

In [58]:
merged_hpv_fce = pd.merge(violhist, fce, how='outer', left_on=['PGM_SYS_ID', 'HPV_DAYZERO_DATE'], right_on=['PGM_SYS_ID','ACTUAL_END_DATE'])

In [105]:
#finding 0's
non_viol = merged_hpv_fce[merged_hpv_fce.HPV_DAYZERO_DATE.isnull() & merged_hpv_fce.ACTUAL_END_DATE.notnull() ]
non_viol['is_violation'] = 0
non_viol = non_viol.filter(['PGM_SYS_ID', 'ACTUAL_END_DATE', 'is_violation'], axis = 1)
non_viol.rename(columns={'ACTUAL_END_DATE': 'HPV_DAYZERO_DATE'}, inplace=True)
#finding 1's
viol = merged_hpv_fce[merged_hpv_fce.HPV_DAYZERO_DATE.notnull()]
viol['is_violation'] = 1
viol = viol.filter(['PGM_SYS_ID', 'HPV_DAYZERO_DATE', 'is_violation'], axis = 1)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [113]:
output = pd.concat([viol,non_viol])
print(merged_hpv_fce.shape, non_viol.shape, viol.shape, output.shape)

(1404806, 34) (1366990, 3) (37782, 3) (1404772, 3)
